# This notebook processes CAFE c2 ocean daily data for building climatologies. Only the last 100 years are used.

In [1]:
# Import packages -----
import pandas as pd
import xarray as xr
import numpy as np
from ipywidgets import FloatProgress
from dateutil.relativedelta import relativedelta

#### Initialise

In [2]:
# Standard naming -----
fields = pd.DataFrame( \
        {'name_CAFE': ['sst', 'patm_t', 'eta_t', 'sss', 'u_surf', 'v_surf', 'mld'],
         'name_std' : ['sst', 'patm',   'eta',   'sss', 'u_s',    'v_s',    'mld']}
                     )
name_dict = fields.set_index('name_CAFE').to_dict()['name_std']

fields

,name_CAFE,name_std
0,sst,sst
1,patm_t,patm
2,eta_t,eta
3,sss,sss
4,u_surf,u_s
5,v_surf,v_s
6,mld,mld


#### Only use last 100 years

In [3]:
# Loop over all paths -----
base = '/OSM/CBR/OA_DCFP/data2/model_output/CAFE/controls/c2/OUTPUT/'
years = range(400,500)

paths = []
for year in years:
    path = base + 'ocean_daily_0' + str(year) + '_01_01.nc'
    paths.append(path)

ds = xr.open_mfdataset(paths, autoclose=True) \
       .drop(['average_T1','average_T2','average_DT','time_bounds',
              'area_t','area_u','geolat_c','geolat_t','ht','mld_sq']) \
       .rename(name_dict)
        
if 'xu_ocean' in ds.dims:
    ds = ds.rename({'xu_ocean':'lon_2','yu_ocean':'lat_2'})
if 'xt_ocean' in ds.dims:
    ds = ds.rename({'xt_ocean':'lon','yt_ocean':'lat'})

/OSM/CBR/OA_DCFP/apps/squ027/anaconda3/envs/scratch_env/lib/python3.6/site-packages/xarray/coding/times.py:132: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy cftime.datetime objects instead, reason: dates out of range
  enable_cftimeindex)
/OSM/CBR/OA_DCFP/apps/squ027/anaconda3/envs/scratch_env/lib/python3.6/site-packages/xarray/coding/variables.py:66: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy cftime.datetime objects instead, reason: dates out of range
  return self.func(self.array[key])


In [4]:
# Use year 2016 as time -----
path = '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/yr2016/mn1/OUTPUT.1/ocean_daily*.nc'
dataset = xr.open_mfdataset(path, autoclose=True)
time_use = xr.concat([dataset.time[:59], dataset.time[60:366]],dim='time')
time_ly = dataset.time[59]

In [5]:
# Make month_day array of month-day -----
m = [str(ds.time.values[i].timetuple()[1]).zfill(2) + '-' for i in range(len(ds.time))]
d = [str(ds.time.values[i].timetuple()[2]).zfill(2) for i in range(len(ds.time))]
md = np.core.defchararray.add(m, d)

# Replace time array with month_day array and groupby -----
ds['time'] = md
clim = ds.groupby('time').mean(dim='time',keep_attrs=True)
clim['time'] = time_use

In [6]:
# Replicate Feb 28th as Feb 29th to deal with leap years -----
clim_ly = clim.copy().sel(time='2016-02-28')
clim_ly['time'] = np.array([time_ly.values])
clim = xr.auto_combine([clim,clim_ly]).sortby('time')

In [7]:
# Save the climatology -----
save_fldr = '/OSM/CBR/OA_DCFP/data/intermediate_products/pylatte_climatologies/'
clim.to_netcdf(save_fldr + 'cafe.c2.ocean.400_499.clim.nc', mode = 'w',
               encoding = {'time':{'dtype':'float','calendar':'JULIAN',
                           'units':'days since 0001-01-01 00:00:00'}})

/OSM/CBR/OA_DCFP/apps/squ027/anaconda3/envs/scratch_env/lib/python3.6/site-packages/dask/array/numpy_compat.py:48: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
